In [ ]:
import pandas as pd

df = pd.read_csv(r'/home.local/aloradi/logs/eval/runs/2025-01-30_00-43-22/test_scores.csv')
df_test = pd.read_csv('/home.local/aloradi/logs/eval/runs/2025-01-30_00-43-22/vpc2025_artifacts/test.csv', sep="|")

df['rel_filepath'] = df['audio_path'].str.extract(r'(B3.*)')
df = df.merge(df_test[['speaker_id', 'rel_filepath', 'gender', 'recording_duration', 'text']], on='rel_filepath', how='left')

In [ ]:
import numpy as np
from speechbrain.utils.metric_stats import EER
import torch
import matplotlib.pyplot as plt

# Compute EER for each enrollment speaker
speakers = df['enrollment_id'].unique()
genders = df['gender'].unique()
eer_results_speaker = {}
eer_results_gender = {}

for speaker in speakers:
    speaker_data = df[df['speaker_id'] == speaker]
    scores = speaker_data['score'].values
    labels = speaker_data['label'].values
    if labels.mean() == 0:
        import ipdb; ipdb.set_trace()
    eer, _ = EER(torch.tensor(scores), torch.tensor(labels))
    eer_results_speaker[speaker.split('_')[-1]] = eer

for gender in genders:
    speaker_data = df[df['gender'] == gender]
    scores = speaker_data['score'].values
    labels = speaker_data['label'].values
    eer, _ = EER(torch.tensor(scores), torch.tensor(labels))
    eer_results_gender[gender] = eer

In [ ]:
# Radar plot
labels = list(eer_results_speaker.keys())
values = list(eer_results_speaker.values())

# Number of variables
num_vars = len(labels)

# Compute angle of each axis
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is a circle, so we need to "complete the loop"
values += values[:1]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
ax.fill(angles, values, color='red', alpha=0.25)
ax.plot(angles, values, color='red', linewidth=2)

# Labels
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)

# show values per speaker
for i in range(num_vars):
    angle_rad = angles[i]
    angle_deg = angle_rad * 180 / np.pi
    ax.text(angle_rad, 1.25, f'{values[i]:.2f}', ha='center', va='center', rotation=angle_deg)

plt.show()